# Dilation Curve Plotter

This notebook is for the easy plotting of multiple dilation curves, from the same sample but varying cooling rates, on the same figure but offset from each other. Plotting dilation curves in this way can sometimes assist with dilatometry analysis and acts as a nice way to present data.

## 1. Input Parameters

The input parameters for this notebook are: **dict_of_data**, **L0**, **N**, **dict_of_offsets**, **sample**.

### Parameter 1: dict_of_data

A python dictionary containing the .asc file names (+ file paths if required) for each dilatometry test. This is used to loop through each dataset. The .asc files should be in the same format used in the '**Transformation_Ts_Calculator.ipynb**' notebook - saving only temperature and change in length. For more information see this notebook.

The layout of a python dictionary is made up of keys and values, and is as follows:

    dictionary = { key1 : value1, key2: value2, ect. }
    
So, for example, let's say each .asc file is saved in a folder called 'data' within this directory, the filepath for these files would be:

    filepath = 'data\'
    
Let's say this folder contains 3 different tests, for 0.1, 1 & 10 °C/s cools, called:

    '0.1.asc'
    '1.asc'
    '10.asc'
    
These files have been provided alongside this notebook for users to view in a folder called '**data**'.
    
The dictionary of data (dict_of_data) should therefore be inputted as:

    dict_of_data = {'0.1': 'data\0.1.asc', '1': 'data\1.asc', '10': 'data\10.asc'}
    
where the dictionary keys are the cooling rate (in string format - i.e., in apostrophes ' ') and the values are the file names (+ file path). Note: the '\data' filepath would not be required if all .asc files are within the same directory as the notebook running this code.

### Parameter 2: L0

The initial length of the sample in metres. This parameter is used to calculate strain from change in length and is usually 0.01 m.

An example of this input would be:

    L0 = 0.01

### Parameter 3: N

The row number at which the cooling step begins. For example, if you have a simple heat, hold and quench cycle, quenching might begin at row number 12000 and should remain the same for every cooling rate if the heating and holding parameters are kept constant. This value will have to be looked up in the raw .asc file beforehand. Basically, the value is used to separate the cooling curve data from the rest of the test.

An example input would be:

    N = 12000

### Parameter 4: dict_of_offsets

A dictionary of the offset values (i.e., the amount each dilation curve will be shifted up the y-axis). It is recommended that each curve should be offset by between 0.002 to 0.006 - but this may need to be adjusted depending on the testing parameters. Bascially, have a play around with these values until the curves are offset by an even amount each.

For example, for the .asc files discussed previously the offset values are:

    dict_of_offsets = {'0.1':0,'1':0.006,'10':0.009}

### Parameter 5: sample

An input for the sample name. This parameter is used in the title of the figure and in naming the .png file saved to the directory.

An example of this input would be:

    sample = 'SA-540, TA=870'

## 2. Example

An example on how to use this notebook is shown in '**dilation_plotter_example.png**'.

## 3. Code

In [6]:
def dilation_plotter(dict_of_data, L0, N, dict_of_offsets, sample):
    # import required packages
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12,8))  
    for r in dict_of_data:
        # read raw dilatometry data, format column names and calculate strain
        data = pd.read_csv(dict_of_data[r], delimiter = r"\s+", header = None, skiprows=4, usecols = range(1,3), error_bad_lines=False, engine='python')
        data.columns = ['T', 'x']
        data['e'] = (data['x']*10**(-6))/L0
        
        # Isolate the cooling curve
        dataset = data[N:]
        Temp = np.array(dataset['T'])
        strain = np.array(dataset['e'])
        
        # offset each dilation up the y-axis
        strain_off = strain + dict_of_offsets[r]
        
        # plot curve
        plt.plot(Temp, strain_off, color = 'k')
        plt.text(max(data['T']), strain_off[0], r, bbox={'facecolor': 'white'})  
    
    # define figure layout
    plt.title(sample+' Dilation Curves', fontsize = 18)
    plt.xlabel('Temperature (\N{DEGREE SIGN}C)', fontsize = 14)
    plt.ylabel('Strain (m/m)', fontsize = 14)
    plt.yticks([])
    plt.savefig(sample+'_Dilation Curves.png')